# Uso de modelos de embeddings de OpenAI

## Instalación y carga de librerías

In [1]:
# !pip install openai
# !pip install tiktoken

In [2]:
from openai import OpenAI
from getpass import getpass
import os
import pandas as pd

In [3]:
# OPENAI_API_KEY = getpass('Enter the secret value: ')
# openai.api_key = OPENAI_API_KEY

## Cargar dataset

In [4]:
df = pd.read_csv('../Platzi_codes/emydb_v11-notebook_5_embeddings_openai-generic-food.csv')

In [ ]:
df.head(2)

## Evaluar cantidad de tokens a procesar

In [6]:
import tiktoken # contabilizar la cantidad de tokens que se enviarían a OpenAI

def num_tokens_from_string(text, encodig_name):
    encoding = tiktoken.get_encoding(encodig_name)
    num_tokens = len(encoding.encode(text))
    return num_tokens


In [7]:
df['total_tokens'] = df['FOOD NAME'].apply(lambda x : num_tokens_from_string(x,'cl100k_base') )

In [ ]:
sum(df['total_tokens'])

## Generando emebeddings

__Esta es la estructura respuesta del modelo de embedding__

```markdown
{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.006929283495992422,
        -0.005336422007530928,
        -4.547132266452536e-05,
        -0.024047505110502243
      ],
    }
  ],
  "model": "text-embedding-3-small",
  "usage": {
    "prompt_tokens": 5,
    "total_tokens": 5
  }
}
```

Estas lineas fue la versión extendida

```python
client = OpenAI()
def get_embedding(in_text, in_model="text-embedding-3-small"):
    temp_text = in_text.replace("\n", " ") # no tengo claro por qué esta linea
    response = client.embeddings.create(input=[temp_text], model=in_model)
    embedding_response = response.data[0].embedding
    return embedding_response
```

En el siguiente script hagola reducción

In [34]:
client = OpenAI()
def get_embedding(in_text, in_model="text-embedding-3-small"):
    in_text = in_text.replace("\n", " ")
    return client.embeddings.create(input=[in_text], model=in_model).data[0].embedding

In [36]:
df['ada_embeddings'] = df['FOOD NAME'].apply(lambda x :get_embedding(x,in_model="text-embedding-3-small"))

In [ ]:
df.head(2)

In [43]:
'''
INFORMATIVO
'''
# display(type(df['ada_embeddings'][0]))
# display(df['ada_embeddings'][0])
# display(len(df['ada_embeddings'][0]))

## Creando datasets para visualizar

Como la data se va a llevar a ~~XXXXembedding~~ se debe formatear

In [ ]:
'''
INFORMATIVO
'''

# df['ada_embeddings'].head(5)

In [ ]:
'''
INFORMATIVO
'''
# type(list(df['ada_embeddings']))

In [ ]:
'''
INFORMATIVO
'''

#list(df['ada_embeddings'])

In [ ]:
'''
INFORMATIVO
'''

# df['ada_embeddings'].apply(type).value_counts()

In [47]:
# Paso 1
df_embeddings = pd.DataFrame(list(df['ada_embeddings']))

In [ ]:
'''
INFORMATIVO
'''

df_embeddings.head(5)

In [54]:
# Paso 2: Guardar los embeddings (df['ada_embeddings']) en formato tsv
df_embeddings.to_csv('embedding_food.tsv',sep='\t',index=False, header=False)

In [55]:
# Creación de un dataframe para la metadata y labels
df[['FOOD NAME','GROUP','SUB GROUP']].to_csv('labels_food.tsv',sep='\t',index=False, header=True)

\# Ir a 
[Embedding Projector](https://projector.tensorflow.org/)

Allí cargar los embedding emydb_v11-notebook_5_embeddings_openai_embedding_food y los lables emydb_v11-notebook_5_embeddings_openai_labels_food